<a target="_blank" href="https://colab.research.google.com/github/opensim-org/opensim-core/blob/main/Bindings/Python/tutorials/Tutorial%207%20-%20Introduction%20to%20OpenSim%20Moco.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 7. Introduction to OpenSim Moco


## 7.1. Objectives

**Introduction**

[OpenSim Moco](https://opensim-org.github.io/opensim-moco-site/) is a software package for creating musculoskeletal simulations using trajectory optimization [[1]](https://doi.org/10.1371/journal.pcbi.1008493). Users define problems by providing an OpenSim model, cost function terms, and additional problem constraints via an easy-to-use scripting interface. Moco rapidly solves trajectory optimization problems via the state-of-the-art direct collocation method.

**Purpose**

The purpose of this tutorial is to demonstrate how to use Moco to build, solve, and analyze a simple trajectory optimization problem using the [OpenSim conda package](https://anaconda.org/opensim-org/opensim) and [Jupyter notebooks](https://jupyter.org/). In this tutorial you will:

*   Set up a new conda environment and install OpenSim with Moco.
*   Learn the basics of Moco's `MocoStudy` interface for defining optimal control problems.
*   Solve Moco's simplest example, the "sliding mass" problem.
*   Plot the results using `matplotlib`.

**Sliding Mass Problem**

You will use Moco to find the optimal trajectory of a point mass starting at rest, translating one meter, and ending at rest, in minimum time.


![sliding_mass](https://drive.google.com/uc?export=download&id=17YTG2WqP6Hzg83OwGyOjqXmyqIWyl34s)

**Local Installation**

You may also run this notebook using a locally installed OpenSim `conda` package. Visit [the OpenSim python scripting Confluence page](https://opensimconfluence.atlassian.net/wiki/spaces/OpenSim/pages/53085346/Scripting+in+Python) for more information on how to install OpenSim in a local Python environment. You will also need to download the resources from Google Drive to the same directory as this notebook. Then, you may skip to Section 7.3 below.

## 7.2. Install `conda` and OpenSim

In the following cell, you will use [condacolab](https://github.com/conda-incubator/condacolab) to install [Miniconda](https://docs.conda.io/en/latest/miniconda.html). If you want to install a different conda distribution (e.g., Anaconda, Mambaforge) check [condacolab's documentation](https://github.com/conda-incubator/condacolab#usage).

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Don't worry if after executing the previous cell you get an error saying that your session has failed. This happens because `condacolab` needs to restart the session for the changes to take effect. Therefore, you have to execute the previous cell before executing any other cell.

Now, execute the following cell to install the OpenSim `conda` package.

In [ ]:
!conda install opensim-org::opensim

## 7.3. Import the OpenSim module

The following cell imports the OpenSim module and, if successful, shows the current version and build date.

In [ ]:
import opensim as osim
osim.GetVersionAndDate()

## 7.4. Create the sliding mass model

First, let's create an OpenSim `Model` representing a "sliding mass": a point mass model with a 1-DOF `SliderJoint` actuated by a `CoordinateActuator`. Moco will use this model to enforce the dynamics defect constraints in the direct collocation problem we will solve below.

In [ ]:
model = osim.Model()
model.setName('sliding_mass')
model.set_gravity(osim.Vec3(0, 0, 0))
body = osim.Body('body', 2.0, osim.Vec3(0), osim.Inertia(0))
model.addComponent(body)

# Allows translation along x.
joint = osim.SliderJoint('slider', model.getGround(), body)
coord = joint.updCoordinate()
coord.setName('position')
model.addComponent(joint)

actu = osim.CoordinateActuator()
actu.setCoordinate(coord)
actu.setName('actuator')
actu.setOptimalForce(1)
model.addComponent(actu)

body.attachGeometry(osim.Sphere(0.05))

model.finalizeConnections()

## 7.5. Create a `MocoStudy`

`MocoStudy` is the most flexible interface for defining optimal control problems in Moco. It encapsulates both a `MocoProblem`, the optimal control problem definition, and a `MocoSolver`, the module which converts the optimal control problem to a nonlinear program using direct collocation.

Run the following code cell to create a new `MocoStudy`, access the underlying `MocoProblem`, and assign the problem the `Model` we created above.

In [ ]:
# Create MocoStudy.
# ================
study = osim.MocoStudy()
study.setName('sliding_mass')

# Define the optimal control problem.
# ===================================
problem = study.updProblem()

# Model (dynamics).
# -----------------
problem.setModel(model)

## 7.6. Define the state bounds and objective function

The sliding mass problem is a boundary value problem (BVP), so we need to set some bounds for the state variables, e.g., the mass position and speed. Run the following cell to set the initial position to 0, the final position to 1, and the initial and final speeds to zero. 

This cell also configures a minimum time objective function using `MocoFinalTimeGoal`. Since we are optimizing time, the final time bounds are set to the interval `[0, 5]` seconds.

In [ ]:
# Bounds.
# -------
# Initial time must be 0, final time can be within [0, 5].
problem.setTimeBounds(osim.MocoInitialBounds(0.), osim.MocoFinalBounds(0., 5.))

# Position must be within [-5, 5] throughout the motion.
# Initial position must be 0, final position must be 1.
problem.setStateInfo('/slider/position/value', osim.MocoBounds(-5, 5),
                     osim.MocoInitialBounds(0), osim.MocoFinalBounds(1))
# Speed must be within [-50, 50] throughout the motion.
# Initial and final speed must be 0. Use compact syntax.
problem.setStateInfo('/slider/position/speed', [-50, 50], [0], [0])

# Applied force must be between -50 and 50.
problem.setControlInfo('/actuator', osim.MocoBounds(-50, 50))

# Cost.
# -----
problem.addGoal(osim.MocoFinalTimeGoal())

## 7.7. Solve the direct collocation problem

Next, we will use `MocoCasADiSolver` to transcribe our optimal control problem to a nonlinear program (NLP) using direct collocation. This `MocoSolver` uses CasADi [[2]](https://doi.org/10.1007/s12532-018-0139-4), an open-source software framework for numerical optimization. Internally, Moco uses CasADi to convert your `MocoProblem` to a direct collocation problem written using CasADi's symbolic and numerical types.

Run the following cell to intialize the `MocoCasADiSolver`, solve the the optimal control problem, and write the solution to a file. Note that if you want to re-run this cell, you will need to reset the Jupyter notebook to clear workspace variables since `study.initCasADiSolver()` cannot be called after a `MocoSolver` has already been initialized.

In [ ]:
# Configure the solver.
# =====================
solver = study.initCasADiSolver()
solver.set_num_mesh_intervals(100)

# Now that we've finished setting up the study, print it to a file.
study.printToXML('sliding_mass.omoco')

# Solve the problem.
# ==================
solution = study.solve()

solution.write('sliding_mass_solution.sto')

You should now have `sliding_mass.omoco` and `sliding_mass_solution.sto` in your files.

## 7.8. Plot the solution

Finally, let's plot the solution. After solving the problem above, `MocoStudy` returned a `MocoSolution` containing the state and control trajectories which we can access directly in Python using `getStateMat()` and `getControlMat()`, respectively.

In [ ]:
import matplotlib.pyplot as plt

time = solution.getTimeMat()
position = solution.getStateMat('/slider/position/value')
speed = solution.getStateMat('/slider/position/speed')
actuator = solution.getControlMat('/actuator')

# Create six subplots, with 2 rows and 3 columns.
fig, axs = plt.subplots(1, 3, figsize=(12, 5))
fig.suptitle('Sliding Mass Solution')

# Plot the knee angles on the first subplot.
axs[0].plot(time, position, label='position', lw=4)
axs[0].set_ylabel('position (m)')
axs[0].set_xlabel('time (s)')

axs[1].plot(time, speed, label='speed', lw=4)
axs[1].set_ylabel('speed (m/s)')
axs[1].set_xlabel('time (s)')

axs[2].plot(time, actuator, label='actuator', lw=4)
axs[2].set_ylabel('actuator force (N)')
axs[2].set_xlabel('time (s)')

for ax in axs:
    ax.set_xlim(0, time[-1])
    ax.grid()

plt.tight_layout()

**Questions**

*  Describe the state and control trajectories returned after solving the sliding mass problem. Given the optimal control problem defined above, does this solution make sense to you? Explain. 

## 7.9. Conclusion

In this tutorial you used [condacolab](https://github.com/conda-incubator/condacolab) to install [Miniconda](https://docs.conda.io/en/latest/miniconda.html) in [Google Colab](https://colab.research.google.com/?hl=en). Then, you installed Conda [OpenSim](https://opensim.stanford.edu/) with Moco into the Miniconda environment. You used the OpenSim API to build a "sliding mass" model, and then used Moco to define an optimal control problem using the `MocoStudy` interface. Finally, you plotted the results using `matplotlib`.

## 7.10. Useful Links

> **OpenSim Website:** https://opensim.stanford.edu/
>
> **OpenSim Moco Website:** https://opensim-org.github.io/opensim-moco-site/
>
> **OpenSim Python Scripting:** https://opensimconfluence.atlassian.net/wiki/spaces/OpenSim/pages/53085346/Scripting+in+Python
>
> **OpenSim API Documentation:** https://simtk.org/api_docs/opensim/api_docs/
>
> **OpenSim Creator Website:** https://opensimcreator.com/
>
> **SimTK Website:** https://simtk.org/projects/opensim
>
> **Biomechanics of Movement Course Videos:** https://www.youtube.com/channel/UCDNGy0KKNLQ-ztcL5h2Z6zA

## 7.11. Acknowledgments

Thanks to [OpenSimColab](https://simtk.org/projects/opencolab) project [[3]](https://doi.org/10.1080/10255842.2022.2104607) for creating the first OpenSim Conda package.

## 7.12. References

> [1] Dembia CL, Bianco NA, Falisse A, Hicks JL, Delp SL (2020) **OpenSim Moco: Musculoskeletal optimal control.** *PLOS Computational Biology* 16(12): e1008493. https://doi.org/10.1371/journal.pcbi.1008493
>
> [2] Andersson JAE, Gillis J, Horn G, Rawlings JB, Diehl M (2019) **CasADi – A software framework for nonlinear optimization
and optimal control.** *Mathematical Programming Computation* 11(1): 1-36. https://doi.org/10.1007/s12532-018-0139-4
>
> [3] Mokhtarzadeh, H., Jiang, F., Zhao, S., & Malekipour, F. (2022). **OpenColab project: OpenSim in Google colaboratory to explore biomechanics on the web.** *Computer Methods in Biomechanics and Biomedical Engineering*, 1–9. https://doi.org/10.1080/10255842.2022.2104607